# Introduction

This fifth notebook shows you how to explore the trust score results in a notebook. If you have not already done so, please run the [fourth notebook](patient-readmission-trust-scan.ipynb) to scan the models for their trust scores.

In this notebook, we will:
1. Load the previously saved trust score reports
2. Convert the results into a dataframe and display them


In [1]:
import numpy as np
import pandas as pd
from pprint import pprint

from certifai.scanner.report_reader import ScanReportReader
from certifai.scanner.report_utils import scores, construct_scores_dataframe
from certifai.scanner.builder import ExplanationType
from IPython.display import display, Markdown

# Loading the Trust Score Reports

To load the reports, we need to know the use case ID ('readmission') and the scan ID.

List the available use cases, and the scans within the 'readmission' use case.

In [2]:
reader = ScanReportReader("reports")
reader.list_usecases()
scans = reader.list_scans('readmission')
data=[[s['date'], ', '.join(s['reportTypes']), s['id']] for s in scans]
df = pd.DataFrame(data, columns=['date', 'evals', 'scan id']).sort_values(by=['date'], ascending=False)
print(df)

              date                                      evals       scan id
0  20201005T121825  fairness, explainability, atx, robustness  5fe6e58129e1
1  20201005T112538                                explanation  7ecad399901a


In [3]:
latest_scan = df[df['evals'].str.contains('fairness') & df['evals'].str.contains('robustness')].iloc[0]
results = reader.load_scan('readmission', latest_scan['scan id'])

# Extract the results

In this section we'll construct dataframes containing the score results and display them.

Print out the fairness results.

In [4]:
df = construct_scores_dataframe(scores('fairness', results), include_confidence=False)
display(df)

,context,type,overall fairness,Feature (age),Group details (15),Group details (25),Group details (35),Group details (45),Group details (5),Group details (55),...,Feature (gender),Group details (Female),Group details (Male),Feature (race),Group details (AfricanAmerican),Group details (Asian),Group details (Caucasian),Group details (Hispanic),Group details (Other),Group details (nan)
mlp (burden),mlp,burden,70.769926,74.396909,0.058918,0.139530,0.077514,0.099075,0.007507,0.067173,...,96.291689,0.076911,0.073275,71.206339,0.076212,0.032217,0.089581,0.078510,0.067697,0.018265
logit (burden),logit,burden,56.444162,56.522626,0.026345,0.232635,0.093680,0.124612,0.000499,0.056315,...,84.492817,0.089148,0.065302,64.472649,0.079077,0.024983,0.097083,0.065481,0.067120,0.014053


For an example of visualizing the fairness results in a chart, see the [fairness metrics notebook](https://raw.githubusercontent.com/CognitiveScale/cortex-certifai-examples/sdk-model-examples/notebooks/fairness_metrics).

Print out the robustness scores.

In [5]:
df = construct_scores_dataframe(scores('robustness', results), include_confidence=True)
display(df)

,context,robustness,robustness lower bound,robustness upper bound
logit,logit,51.885452,48.761420,54.976354
mlp,mlp,50.988814,46.306286,55.818630


Print out the explainability scores.

In [6]:
df = construct_scores_dataframe(scores('explainability', results), include_confidence=False)
display(df)

,context,explainability,Num features (1),Num features (10),Num features (2),Num features (3),Num features (4),Num features (5),Num features (6),Num features (7),Num features (8),Num features (9)
mlp,mlp,96.093750,66.40625,0.0,31.25000,1.5625,0.78125,0.0,0.0,0.0,0.0,0.0
logit,logit,94.453125,53.90625,0.0,39.84375,6.2500,0.00000,0.0,0.0,0.0,0.0,0.0
